In [1]:
# Project Libraries
import data

# Generic Libraries
import numpy as np
import pandas as pd

Data Frames:
- Orderbook with previous operations
- Liquidity pools history (deliverable)
- Trade History (deliverable) 

In [18]:
# ----- Useful dataframes

def dataframes():
    """
    L

    Parameters
    ----------

    L

    Returns
    -------

    L
    
    """
    
    # Informative dataframe
    df_info = pd.DataFrame({
        'bid_volume': [0.0001],
        'ask_volume': [0.0001],
        'commission': [0.000025]
    })

    # Oders dataframe
    df_orders = pd.DataFrame(columns = ['timestamp', 'order_id', 'side', 'price', 'order_amount', 'symbol'])

    # Trades dataframe
    df_trades = pd.DataFrame(columns = ['timestamp', 'order_id', 'trade_id', 'side', 'price', 'symbol', 'filled_amount', 'order_amount', 'fee'])

    # Inventory dataframe
    df_inventory = pd.DataFrame({
        'timestamp': [0],
        'base (BTC)': [5],
        'quote (USDT)': [500000]
    })

    # Rebalance dataframe
    df_rebalances = pd.DataFrame(columns = ['timestamp', 'sent', 'received', 'price', 'fee'])

    return df_info, df_orders, df_trades, df_inventory, df_rebalances

#df_info, df_orders, df_trades, df_inventory, df_rebalances = fn.dataframes()
df_info, df_orders, df_trades, df_inventory, df_rebalances = dataframes()

In [19]:
df_orders

,timestamp,order_id,side,price,order_amount,symbol


In [10]:
try:
    df_orders['order_id'].iloc[-1] + 1
except:
    print(1)

1


In [13]:
def order_id(col):
    if col == 'id':
        return 1
    elif col == 'index':
        try:
            return df_orders.index[0] + 1
        except:
            return 0

In [28]:
order_id('index')

0

In [26]:
def order_id(col):
    if col == 'order_id':
        try:
            return df_orders['order_id'].iloc[-1] + 1
        except:
            return 1
    if col == 'index':
        try:
            return df_orders.index[0] + 1
        except:
            return 0

In [17]:
order_placement(df = df_bitfinex_tob,
                ts = 0,  
                    position = "buy")
order_placement(df = df_bitfinex_tob, 
                ts = 0,
                    position = "sell")
df_orders

,timestamp,order_id,side,price,order_amount,symbol
0,2021-07-05 13:06:00+00:00,None,buy,28282.0,0.0001,BTC/USDT
1,2021-07-05 13:06:00+00:00,1,sell,28282.0,0.0001,BTC/USDT


In [15]:
# ----- Order Placement Function
def order_placement(df, ts, position):
    i = order_id('index')
    if position == 'buy':
        df_orders.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
        df_orders.at[i, 'order_id'] = order_id(col = 'id')
        df_orders.at[i, 'side'] = 'buy' 
        df_orders.at[i, 'price'] = df['bid'].iloc[ts]
        df_orders.at[i, 'order_amount'] = 0.0001
        df_orders.at[i, 'symbol'] = 'BTC/USDT'
    else:
        df_orders.at[i, 'timestamp'] = df['timestamp'].iloc[ts]
        df_orders.at[i, 'order_id'] = order_id(col = 'id')
        df_orders.at[i, 'side'] = 'sell'
        df_orders.at[i, 'price'] = df['bid'].iloc[ts]       
        df_orders.at[i, 'order_amount'] = 0.0001
        df_orders.at[i, 'symbol'] = 'BTC/USDT'

In [7]:
# Explanation Missing
def position_valuation(size_variation: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation > 0:
            current_position = 0
        elif price_variation == 0:
            if size_variation > 0:
                current_position = constant_position_base
            else:
                if size_variation <= constant_position_base:
                    current_position = constant_position_base - size_variation
                else:
                    current_position = 0
        else:
            current_position = constant_position_base
    # Bid
    if operation == 'bid':
        if price_variation < 0:
            current_position = 0
        elif price_variation == 0:
            if size_variation > 0:
                current_position = constant_position_base
            else:
                if size_variation <= constant_position_base:
                    current_position = constant_position_base - size_variation
                else:
                    current_position = 0
        else:
            current_position = constant_position_base
    return current_position

In [8]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})


In [52]:
order_placement(df = df_bitfinex_tob,
                ts = 0,  
                    position = "buy")
order_placement(df = df_bitfinex_tob, 
                ts = 0,
                    position = "sell")
df_orders

,timestamp,order_id,side,price,order_amount,symbol
0,2021-07-05 13:06:00+00:00,None,buy,28282.0,0.0001,BTC/USDT
1,2021-07-05 13:06:00+00:00,None,sell,28323.0,0.0001,BTC/USDT


In [17]:
df_bitfinex_tob

,timestamp,ask_size,ask,bid,bid_size
0,2021-07-05 13:06:00+00:00,0.100600,28283.0,28282.0,0.338172
1,2021-07-05 13:07:00+00:00,2.792849,28325.0,28323.0,0.594797
2,2021-07-05 13:08:00+00:00,0.403360,28315.0,28309.0,0.093481
3,2021-07-05 13:09:00+00:00,0.148733,28333.0,28329.0,0.339650
4,2021-07-05 13:10:00+00:00,0.003524,28341.0,28336.0,3.807731
...,...,...,...,...,...
56,2021-07-05 14:02:00+00:00,3.550600,28292.0,28288.0,0.446627
57,2021-07-05 14:03:00+00:00,0.385010,28301.0,28295.0,0.600127
58,2021-07-05 14:04:00+00:00,0.009931,28352.0,28340.0,0.357682
59,2021-07-05 14:05:00+00:00,0.445649,28361.0,28355.0,0.006000


In [77]:
order_placement(df = df_bitfinex_tob, 
                    ts = 0, 
                    position = "buy")
order_placement(df = df_bitfinex_tob, 
                    ts = 0, 
                    position = "sell")

volume_fill_ask =  0

iteration = 1

filled_amount_bid = position_valuation(
    size_variation  = df_bitfinex_tob['bid_size'].iloc[iteration] - df_bitfinex_tob['bid_size'].iloc[iteration - 1],
    price_variation = df_bitfinex_tob['bid'].iloc[iteration] - df_orders[df_orders['side'] == 'buy'].iloc[0]['price'],
    operation = 'bid',
    constant_position_base = df_orders[df_orders['side'] == 'buy'].iloc[0]['order_amount'] - volume_fill_ask
    )

volume_fill_ask = volume_fill_ask + filled_amount_bid
buy_size_whole = df_orders[df_orders['side'] == 'buy'].iloc[0]['order_amount']

if volume_fill_ask == buy_size_whole:
    order_placement(df = df_bitfinex_tob, 
                    i = iteration, 
                    position = "buy")


In [7]:
df_orders

,timestamp,order_id,side,price,order_amount,symbol


In [21]:
# ----- Data process
df_bitfinex_tob = ob.copy()
# - Trade

for i in range(len(df_orders) - 2):
    df_trades.at[i, 'timestamp'] = df_orders['timestamp'].iloc[i]
    df_trades.at[i, 'order_id'] = df_orders['order_id'].iloc[i]
    df_trades.at[i, 'side'] = df_orders['side'].iloc[i]
    df_trades.at[i, 'price'] = df_orders['price'].iloc[i]
    df_trades.at[i, 'symbol'] = df_orders['symbol'].iloc[i]
    df_trades.at[i, 'order_amount'] = df_orders['order_amount'].iloc[i]

    if df_orders['price'].iloc[i + 2] != df_orders['price'].iloc[i]:
        df_trades.at[i, 'trade_id'] = i + 1
        df_trades.at[i, 'filled_amount'] = df_orders['order_amount'].iloc[i]
        df_trades.at[i, 'fee'] = df_trades['price'].iloc[i]*df_trades['filled_amount'].iloc[i]*df_info['commission'].iloc[0]

    elif df_orders['price'].iloc[i + 2] == df_orders['price'].iloc[i]:

        for j in range(np.count_nonzero(df_orders['price'].iloc[i:i+3] == df_orders['price'].iloc[i])):
            df_trades.at[i, 'trade_id'] = j + 1
            df_trades.at[i, 'filled_amount'] = df_orders['order_amount'].iloc[i]
            df_trades.at[i, 'fee'] = df_trades['price'].iloc[i]*df_trades['filled_amount'].iloc[i]*df_info['commission'].iloc[0]

# --- 3rd part

# - Inventory



# --- 4th part

# Rebalances



# --- 5th Part

# - Results

df_orders

,timestamp,order_id,side,price,order_amount,symbol
0,2021-07-05 13:06:00+00:00,1,buy,28282.0,0.0001,BTC/USDT
1,2021-07-05 13:06:00+00:00,2,sell,28283.0,0.0001,BTC/USDT
2,2021-07-05 13:07:00+00:00,3,buy,28323.0,0.0001,BTC/USDT
3,2021-07-05 13:07:00+00:00,4,sell,28325.0,0.0001,BTC/USDT
4,2021-07-05 13:08:00+00:00,5,buy,28309.0,0.0001,BTC/USDT
...,...,...,...,...,...,...
117,2021-07-05 14:04:00+00:00,118,sell,28352.0,0.0001,BTC/USDT
118,2021-07-05 14:05:00+00:00,119,buy,28355.0,0.0001,BTC/USDT
119,2021-07-05 14:05:00+00:00,120,sell,28361.0,0.0001,BTC/USDT
120,2021-07-05 14:06:00+00:00,121,buy,28354.0,0.0001,BTC/USDT


In [22]:
df_trades

,timestamp,order_id,trade_id,side,price,symbol,filled_amount,order_amount,fee
0,2021-07-05 13:06:00+00:00,1,1,buy,28282.0,BTC/USDT,0.0001,0.0001,0.000071
1,2021-07-05 13:06:00+00:00,2,2,sell,28283.0,BTC/USDT,0.0001,0.0001,0.000071
2,2021-07-05 13:07:00+00:00,3,3,buy,28323.0,BTC/USDT,0.0001,0.0001,0.000071
3,2021-07-05 13:07:00+00:00,4,4,sell,28325.0,BTC/USDT,0.0001,0.0001,0.000071
4,2021-07-05 13:08:00+00:00,5,5,buy,28309.0,BTC/USDT,0.0001,0.0001,0.000071
...,...,...,...,...,...,...,...,...,...
115,2021-07-05 14:03:00+00:00,116,116,sell,28301.0,BTC/USDT,0.0001,0.0001,0.000071
116,2021-07-05 14:04:00+00:00,117,117,buy,28340.0,BTC/USDT,0.0001,0.0001,0.000071
117,2021-07-05 14:04:00+00:00,118,118,sell,28352.0,BTC/USDT,0.0001,0.0001,0.000071
118,2021-07-05 14:05:00+00:00,119,119,buy,28355.0,BTC/USDT,0.0001,0.0001,0.000071


In [ ]:
def trade_trigger()

In [90]:
# Liquidity Pool
initial_liquidity_base  = 5 
initial_liquidity_quote = 5
liquidity_pool = pd.DataFrame(
    np.reshape( np.array([ initial_liquidity_base, initial_liquidity_quote ]), [1,-1] ), 
               columns = [ 'Base', 'Quote' ], index = [ ob.index.values[0] ]
)
liquidity_pool

,Base,Quote
2021-07-05 13:06:00,5,5


In [61]:
# Explanation Missing
def position_valuation(size: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation >= 0:
            if ( constant_position_base - size ) < 0:
                current_position = 0
            else:
                current_position = constant_position_base - size
        else:
            current_position = constant_position_base
    # Bid
    if operation == 'bid':
        if price_variation <= 0:
            if ( constant_position_base - size ) < 0:
                current_position = 0
            else:
                current_position = constant_position_base - size
        else:
            current_position = constant_position_base
    return current_position

In [62]:
for i in range(1,len(ob)):
    try:
        position_valuation( size = ob['ask_size'].iloc[i], price_variation = ob['ask_var'].iloc[i],
                            operation = np.repeat( 'ask', len(ob) )[i]
                        )
    except:
        print(i)

In [78]:
def order_book_information(order_book, position):
    # Ask
    order_book['ask_var'] = (order_book['ask']-initial_prices['ask'])
    order_book['bid_var'] = (order_book['bid']-initial_prices['bid'])

    order_book['ask_position'] = [ position_valuation( 
        size = order_book['ask_size'].iloc[i], price_variation = order_book['ask_var'].iloc[i], operation = np.repeat( 'ask', len(order_book) )[i]
                    ) for i in range(len(order_book)) ]

    order_book['bid_position'] = [ position_valuation( 
        size = order_book['bid_size'].iloc[i], price_variation = order_book['bid_var'].iloc[i], operation = np.repeat( 'bid', len(order_book) )[i]
                    ) for i in range(len(order_book)) ]
    return order_book


In [ ]:
def trade_trigger( position,constant_position_base: float = .0001 ):
    if position != constant_position_base:
        # Activate Trade Function

In [ ]:
def token_request(liquidity_pool)

In [77]:
order_book_information(ob, 0)

,ask_size,ask,bid,bid_size,ask_var,ask_position,bid_var,bid_position
index,,,,,,,,
2021-07-05 13:06:00+00:00,0.100600,28283.0,28282.0,0.338172,0.0,0.0,0.0,0.0000
2021-07-05 13:07:00+00:00,2.792849,28325.0,28323.0,0.594797,42.0,0.0,41.0,0.0001
2021-07-05 13:08:00+00:00,0.403360,28315.0,28309.0,0.093481,32.0,0.0,27.0,0.0001
2021-07-05 13:09:00+00:00,0.148733,28333.0,28329.0,0.339650,50.0,0.0,47.0,0.0001
2021-07-05 13:10:00+00:00,0.003524,28341.0,28336.0,3.807731,58.0,0.0,54.0,0.0001
...,...,...,...,...,...,...,...,...
2021-07-05 14:02:00+00:00,3.550600,28292.0,28288.0,0.446627,9.0,0.0,6.0,0.0001
2021-07-05 14:03:00+00:00,0.385010,28301.0,28295.0,0.600127,18.0,0.0,13.0,0.0001
2021-07-05 14:04:00+00:00,0.009931,28352.0,28340.0,0.357682,69.0,0.0,58.0,0.0001


In [ ]:
def liquidity_verification()

In [ ]:
def trade()

In [40]:
# Market Making Conditions
constant_position_base = 0.0001
liquidity_pool_base  = []
liquidity_pool_quote = []

numpy.datetime64('2021-07-05T13:06:46.000000000')